# Kernel ridge regression on GP datasets 2

We want to show that even on n-dimensional GP datasets, kernel ridge regression with a polynomial kernel does not learn functions as smooth as neural networks do.

This has proven more difficult than expected: when the KRR is not regularized and the degree is high, there is so much numerical instability that we cannot even get low training error. But when we regularize or decrease the degree, the functions _are_ smooth, often even smoother (as measured by `path_length_f`) than the original GP.

This notebook contains:

- comparison of measures between KRR and NNs, normalized by GP ground truth
- in-notebook experiments with polynomials of higher degree (10 to 50)

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0302_gp_krr/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

### Measures of KRR

In [ ]:
ms_kr = pd.read_feather("measures.feather")
ms_kr_2 = pd.read_feather("measures_alpha0.feather")
ms_kr_3 = pd.read_feather("measures_gamma1.feather")
ms_kr = pd.concat([ms_kr, ms_kr_2, ms_kr_3], ignore_index=True)
ms_kr = smooth.analysis.expand_dataset_columns(ms_kr)
smooth.analysis.remove_constant_columns(ms_kr, verbose=True)
ms_kr.rename(columns={"path_length_f_test": "path_length_f"}, inplace=True)
ms_kr.head()

### Measures of shallow relu neural networks

In [ ]:
ms_nn = pd.read_feather("../0302_gp_nn/measures.feather")
ms_nn_64 = pd.read_feather("../0303_gp_nn/measures.feather")

ms_nn = pd.concat([ms_nn, ms_nn_64], sort=False)
ms_nn = smooth.analysis.expand_dataset_columns(ms_nn)

smooth.analysis.remove_constant_columns(ms_nn, verbose=True)
print(len(ms_nn))
ms_nn.head()

### Measures of the original GP model

In [ ]:
ms_gp = pd.read_feather("measures_gp.feather")
ms_gp.head()

Normalized measures - divided by the corresponding measure of GP

In [ ]:
def normalize(df1, df2, join_col, cols):
    assert set(cols + [join_col]).issubset(set(df1.columns))
    df = pd.merge(df1, df2[cols + [join_col]], on=join_col, suffixes=("", "_0"))
    for col in cols:
        df[col + "_n"] = df[col] / df[col + "_0"]
        del df[col + "_0"]
    return df

measure_cols = ["train_loss", "test_loss", "path_length_f"]
ms_kr = normalize(ms_kr, ms_gp, "dataset", measure_cols)
ms_nn = normalize(ms_nn, ms_gp, "dataset", measure_cols)

### Comparison plots

Note that the data is normalized by the GP measures. Also, it is aggregated along three different seeds; the results seem to be fairly consistent.

In [ ]:
def plot_compare(groups, filter_f: None):
    filter_f = filter_f or (lambda df: df)
    l = []
    for group_name, group in groups:
        for name, ms_cur in group:
            ms_cur = ms_cur.copy()
            ms_cur.loc[:, "source"] = name
            ms_cur.loc[:, "group"] = group_name
            l.append(ms_cur)

    ms_all = pd.concat(l, sort=False)
    ms_all = filter_f(ms_all)
    ms_all = ms_all.loc[
        (ms_all["dim"] == dim)
#         & (ms_all["seed"] == 1)
        & (ms_all["lengthscale"] == ms_all["dim"])
    ]
    
    for measure in ["train_loss_n", "test_loss_n", "path_length_f_n"]:
        grid = sns.relplot(
            data=ms_all,
            x="samples_train",
            y=measure,
            hue="source",
            col="group",
            kind="line",
        )
        ax = grid.axes[0][0]
        ax.set_xscale("log")
        if measure in ["train_loss_n", "test_loss_n",
                      "path_length_f_n"
                      ]:
            ax.set_yscale("log")
        if measure in ["path_length_f_n"]:
            ax.set_ylim(0.03, 30)
        plt.show()

kr_group = []
for deg in range(1, 6):
    kr_group.append((
        "krr, deg={}".format(deg),
        ms_kr.loc[(ms_kr["degree"] == deg) & (ms_kr["alpha"] == 0.000) & (ms_kr["gamma"] == 1.)],
#         ms_kr.loc[(ms_kr["degree"] == deg) & (ms_kr["alpha"] == 0.000)],
    ))
# kr_group.append(("gp", ms_gp))

nn_group = []
for hs in sorted(ms_nn["hidden_size"].unique()):
    nn_group.append((
        "nn, hs={:02}".format(hs),
        ms_nn.loc[ms_nn["hidden_size"] == hs],
    ))
# nn_group.append(("gp_nn", ms_gp))

def filter_f(ms):
    return ms.loc[
        (ms["dim"] == dim)
#         & (ms_all["seed"] == 1)
        & (ms["lengthscale"] == ms["dim"])
    ]

for dim in sorted(ms_nn["dim"].unique()):
    display(IPython.display.Markdown("### dim = {}".format(dim)))
    plot_compare([("krr", kr_group), ("nn", nn_group)], filter_f)

### Computing ground truth measures

In [ ]:
path = "./measures_gp.feather"
if os.path.isfile(path):
    raise FileExistsError("We have already computed this")

ms_all = pd.concat([ms_kr, ms_gp], sort=False)
ms_gp_l = []

for dim in tqdm.notebook.tqdm(sorted(ms_all["dim"].unique())):
    for seed in tqdm.notebook.tqdm(sorted(ms_all["seed"].unique()), leave=False):
        for lengthscale_coef in tqdm.notebook.tqdm([0.3, 1.0], leave=False):
            lengthscale = float(dim) * lengthscale_coef
            dataset0 = smooth.datasets.from_name("gp-{}-{}-{}-1000".format(dim, seed, lengthscale))
            for samples in tqdm.notebook.tqdm(sorted(ms_all["samples_train"].unique()), leave=False):
                dataset0.gp_model.set_XY(dataset0.x_train[:samples], dataset0.y_train[:samples])
                model = smooth.measures.GPModel(dataset0.gp_model)
                m = smooth.measures.get_measures_no_gradient(model, dataset0.subset(samples, keep_test_set=True))
                m.update(
                    dim=dim,
                    seed=seed,
                    samples_train=samples,
                    lengthscale=lengthscale,
                )
                ms_gp_l.append(m)

ms_gp = pd.DataFrame()
ms_gp["dataset"] = ("gp-" + ms_gp["dim"].map(str)
                    + "-" + ms_gp["seed"].map(str)
                    + "-" + ms_gp["lengthscale"].map(str)
                    + "-" + ms_gp["samples_train"].map(str)
                   )
ms_gp.to_feather(path)

## High-degree polynomials

How do the results change when we use high-degree polynomials (degree 10, 20, 30, 40, 50)?

In [ ]:
ms_gp = pd.DataFrame(ms_gp_l[40:])
ms_gp["dataset"] = ("gp-" + ms_gp["dim"].map(str)
                    + "-" + ms_gp["seed"].map(str)
                    + "-" + ms_gp["lengthscale"].map(str)
                    + "-" + ms_gp["samples_train"].map(str)
                   )
ms_gp.to_feather("measures_gp.feather")

In [ ]:
%%time
path = "./measures_high_degree.feather"
if os.path.isfile(path):
    raise FileExistsError("We have already computed this")

import sklearn.kernel_ridge
import warnings

ms_krr_l = []

samples_l = np.logspace(np.log10(10), np.log10(1000), 10).round().astype(int)
seed = 1

for dim in tqdm.notebook.tqdm([2, 8, 32, 128], desc="dim"):
    for alpha in tqdm.notebook.tqdm([1., 0.], leave=False, desc="alpha"):
        for gamma in tqdm.notebook.tqdm([1., None], leave=False, desc="gamma"):
            lengthscale = float(dim)
            dataset0 = smooth.datasets.from_name("gp-{}-{}-{}-1000".format(dim, seed, lengthscale))
            for degree in tqdm.notebook.tqdm([10, 20, 30, 40, 50], leave=False, desc="degree"):
                for samples in tqdm.notebook.tqdm(samples_l, leave=False, desc="samples"):
                    krr = sklearn.kernel_ridge.KernelRidge(
                        alpha=alpha,
                        kernel="poly",
                        degree=degree,
                        coef0=1,
                        gamma=gamma,
                    )
                    dataset = dataset0.subset(samples, keep_test_set=True)
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        try:
                            krr.fit(dataset.x_train, dataset.y_train)
                            m = smooth.train_kernel_models.measure_krr(krr, dataset)
                        except ValueError:
                            continue

                    m.update(
                        dim=dim,
                        seed=seed,
                        alpha=alpha,
                        gamma=gamma,
                        degree=degree,
                        samples_train=samples,
                        lengthscale=lengthscale,
                    )
                    ms_krr_l.append(m)
                #     y_pred = krr.predict(dataset.x_test)
                #     break

pd.DataFrame(ms_krr_l).to_feather("measures_high_degree.feather")

In [ ]:
ms_gp

In [ ]:
ms_gp = pd.read_feather("measures_gp.feather")
len(ms_gp)

### Ground-truth measures - the GP is the model

In [ ]:
ms_gp_l = []
for dim in tqdm.notebook.tqdm([2, 8, 32, 128], desc="dim"):
    lengthscale = float(dim)
    dataset0 = smooth.datasets.from_name("gp-{}-{}-{}-1000".format(dim, seed, lengthscale))
    for samples in tqdm.notebook.tqdm(samples_l, leave=False, desc="samples"):
        dataset0.gp_model.set_XY(dataset0.x_train[:samples], dataset0.y_train[:samples])
        model = smooth.measures.GPModel(dataset0.gp_model)
        m = smooth.measures.get_measures_no_gradient(model, dataset0.subset(samples, keep_test_set=True))
        m.update(
            dim=dim,
            seed=seed,
            samples_train=samples,
            lengthscale=lengthscale,
        )
        ms_gp_l.append(m)        

In [ ]:
ms_gp = pd.DataFrame(ms_gp_l)

for measure in ["path_length_f", "path_length_f_train"]:
    grid = sns.relplot(
        data=ms_gp,
        x="samples_train",
        y=measure,
        col="dim",
        kind="line",
    )

    ax = grid.axes[0][0]
    ax.set_xscale('log')
    ax.set_yscale('log')

    plt.show()

### High-degree polynomials vs GP baseline

In [ ]:
ms_krr = pd.DataFrame(ms_krr_l)
ms_krr.rename(columns={"path_length_f_test": "path_length_f"}, inplace=True)
ms_krr = ms_krr.loc[(ms_krr["gamma"] == 1) & (ms_krr["dim"] > 0)]

ms_gp["degree"] = 0
ms_gp["alpha"] = 0

for measure in ["train_loss", "test_loss", "path_length_f", "path_length_f_train"]:
    ms_cur = pd.concat([ms_krr, ms_gp], sort=False)
    grid = sns.relplot(
        data=ms_cur,
        x="samples_train",
        y=measure,
        hue="degree",
        style="alpha",
        col="dim",
        kind="line",
        col_wrap=2,
        palette=smooth.analysis.make_palette(ms_cur["degree"].unique()),
    )
    ax = grid.axes[0]
    ax.set_xscale('log')
    ax.set_yscale('log')

    plt.show()